In [1]:
import pandas as pd

In [2]:
productDF = pd.read_csv('../data/lazada/product.csv')
reviewDF = pd.read_csv('../data/lazada/review.csv')

In [3]:
# extract anonymous review
nonAnonymousReviewDF = reviewDF[reviewDF['user_fk'] != 0]
nonAnonymousReviewDF.sample(5)

,comment,rating,sku,time,user,user_fk
82555,แพ็คสินค้าดี+ส่งของเร็ว ของใช้งานได้ดีตามคุณส...,4,WA038ELAA3QH3BANTH,2017-02-01 23:28:46,สุวัฒน์ ว.,1151993
48656,NaN,1,CH928FAAADH5WMANTH,2017-09-01 08:05:23,ปราณี ทุลภิรมย์,11046374
61403,ดีครับ,5,SE934HLBU6FHANTH,2016-04-22 22:19:23,อัครพงศ์ ศรีปฐมสายชล,3472789
102867,ปิดได้พอดี แต่ไม่สนิทชิดขอบด้านบน เอานิ้วเลื่อ...,5,FO203ELAT0YIANTH,2017-02-10 04:17:17,เฉลิมนาถ บ.,2554501
24418,NaN,3,MA681HLAA7H2O1ANTH,2017-03-07 17:59:19,อรรถพร รื่นภิรมย์,4640500


In [4]:
# select only data complementing each other
productWithReviewDF = productDF[productDF['sku'].isin(nonAnonymousReviewDF['sku'])].copy()
reviewWithProductDF = nonAnonymousReviewDF[nonAnonymousReviewDF['sku'].isin(productDF['sku'])].copy()

In [5]:
print('shape of product df : {}'.format(productWithReviewDF.shape))
print('shape of review df : {}'.format(reviewWithProductDF.shape))

shape of product df : (7392, 4)
shape of review df : (76080, 6)


In [6]:
reviewWithProductDF.shape

(76080, 6)

In [7]:
reviewWithProductCoreDF = reviewWithProductDF.drop(['comment', 'user'], axis=1)
reviewWithProductCoreDF.sample(5)

,rating,sku,time,user_fk
35303,3,UN355ELAA7LEZKANTH,2017-09-11 15:36:30,1064718
88747,5,UN355ELAA4RTXCANTH,2017-06-11 02:30:12,8545598
15111,4,AU889ELAA4AYDSANTH,2016-12-24 11:48:23,283028
114544,3,KZ711ELAA4V2VDANTH,2017-06-09 10:43:20,1210619
121889,3,UN355ELAA7L931ANTH,2017-04-17 08:51:40,8751058


In [8]:
productWithReviewCoreDF = productWithReviewDF.drop(['url'], axis=1)
productWithReviewCoreDF.sample(5)

,name,price,sku
14342,Gopro Gopro/SJCam/Xiaomi/YI 360° Rotary Nut Se...,299.0,GO441ELAA4PW3VANTH
552,Samsung ชุดชาร์จ Samsung Fast Charger อแดปเตอร...,690.0,SA861ELAA5L5GNANTH
13098,มู่ลี่ประตู 032 ขนาด 80 X 200 ซม. (ราคาพิเศษ),260.0,WS056HL89ICWANTH
1359,KitchenMarks ที่ลับมีดอัตโนมัติ ที่ลับมีด ที่ล...,165.0,KI576HLAAE20JSANTH
10581,RF Furniture ตู้เสื้อผ้า 80 cm รุ่น WK002 ( สี...,1990.0,RF271HLAA65P00ANTH


In [9]:
# export
reviewWithProductCoreDF.to_csv('../data/lazada/clean/review.csv', index=False, encoding='utf-8')
productWithReviewCoreDF.to_csv('../data/lazada/clean/product.csv', index=False, encoding='utf-8')